In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps
from api_keys import g_key

In [3]:
# Import weather data and create a dataframe
data = "output_data/WeatherPy_Log.csv"
weather_data_df= pd.read_csv(data)
weather_data_df

,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
0,Hobart,AU,2163355,-42.88,147.33,53.60,81,90,4.70
1,Punta Arenas,CL,3874787,-53.15,-70.92,48.20,45,75,20.80
2,Rikitea,PF,4030556,-23.12,-134.97,73.27,80,3,13.65
3,Comodoro Rivadavia,AR,3860443,-45.87,-67.50,55.40,37,0,17.22
4,Saskylakh,RU,2017155,71.92,114.08,20.86,97,85,7.00
...,...,...,...,...,...,...,...,...,...
552,Ellisras,ZA,1005768,-23.66,27.74,86.63,14,9,5.12
553,Raudeberg,NO,3146487,61.99,5.14,44.60,87,75,19.46
554,Berlevåg,NO,780687,70.86,29.09,30.20,100,75,1.12
555,Springfield,US,4409896,37.22,-93.30,55.99,93,90,11.41


In [15]:
# Create a heat layer map showing the humidity for the cities in the data.
gmaps.configure(api_key=g_key)
weather_data_df = weather_data_df.dropna()
locations = weather_data_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Narrow the list of vacation hotel locations according to weather criteria, find the hotel information
# for each location and filter out locations with no hotels.
Hotel_df = weather_data_df.loc[(weather_data_df["Humidity"]<70)& (weather_data_df["Max Temperature"]<55)& (weather_data_df["Max Temperature"]
                                       >25) & (weather_data_df["Wind Speed"]<5) & (weather_data_df["Cloudiness (%)"]<10)]
Hotel_Name_List = []
locations = Hotel_df[["Latitude", "Longitude"]].astype(float)
for x in range (len(locations)):
    lat=locations.iloc[x,0]
    long=locations.iloc[x,1]
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
            "location": f"{lat},{long}",
            "radius": 5000,
            "keyword" : "hotel",
            "type": "hotel",
            "key": g_key,
    }
    response = requests.get(base_url, params=params).json()
    results=response["results"]
    try: 
        Hotel_Name = results[1]["name"]  
        Hotel_Name_List.append(Hotel_Name)
    except:
        No_Hotel = f"No Hotel near {lat}, {long}"
        Hotel_Name_List.append(No_Hotel)
print(Hotel_Name_List)
Hotel_df["Hotel Name"] = Hotel_Name_List
Hotel_df


['Urumqi Bintuan Grand Hotel', 'Olymp Krokus', 'No Hotel near 32.5, 62.5', 'Tianma Hotel', 'Hotel Hillside Karakol']


/Users/imacmattimacmatt/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed,Hotel Name
62,Ürümqi,CN,1529102,43.80,87.60,46.40,39,0,2.24,Urumqi Bintuan Grand Hotel
108,Angren,UZ,1514581,41.02,70.14,52.61,38,0,1.39,Olymp Krokus
236,Farah,AF,1142263,32.50,62.50,48.31,44,0,2.84,"No Hotel near 32.5, 62.5"
265,Kumul,CN,1529484,42.80,93.45,44.89,54,0,0.20,Tianma Hotel
278,Karakol,KG,1528121,42.49,78.39,45.57,40,0,3.71,Hotel Hillside Karakol


In [22]:
# Map the top hotels onto a layer in google maps.

Hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content= [f"Hotel Name: {name} " for name in Hotel_Name_List]
)
fig = gmaps.figure()
fig.add_layer(Hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add the Hotel marker layer on top of the heat map.
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(Hotel_layer)
fig



Figure(layout=FigureLayout(height='420px'))